# 🧠 Quantifying Neural Responses — Simulating and Visualizing Neural Time Series Data

## Introduction

In neuroscience, we often work with **time series data**, where we record how voltage (or another signal) changes over time.  
For example, EEG and MEG measure voltage fluctuations across multiple sensors — each sensor gives a separate time series.

Before we analyze *real* data, it’s very helpful to **simulate** signals.  
Simulation lets us:

- **Understand** the structure of neural data  
- **Visualize** how signals look in the time and frequency domains  
- **Control** exactly what goes into the data (ground truth)  
- **Experiment safely** before working with complex datasets  

In this notebook, we will simulate different types of signals and visualize them in both the time and frequency domains.


## 1. Representing Neural Data in Python

Neural data are typically stored as a **2D array**, where each **row corresponds to one channel** and each **column corresponds to a time point**:

```
n_channels × n_timepoints
```

Each row is one continuous voltage trace recorded from one sensor.

To represent time digitally, we need to define **sample points** — discrete “ticks” where the signal will be measured.  
The number of samples per second is defined by the **sampling rate** (e.g., 1000 Hz means 1000 samples per second).

Using NumPy, we can create these time points with `np.arange(0, duration, 1/sampling_rate)`,  
which gives evenly spaced points separated by `1/sampling_rate` seconds.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 250  # crisper figures

# Simulation parameters
sampling_rate = 1000  # Hz
duration = 2.0        # seconds
n_channels = 2        # two example channels

# Time vector: one sample every 1/sampling_rate seconds
time = np.arange(0, duration, 1 / sampling_rate)
print(f"Time array shape: {time.shape}")

# Initialize array (channels × timepoints)
data = np.zeros((n_channels, len(time)))
print(f"Data shape: {data.shape}")

# data[c, :] → signal from channel c over time
# data[:, t] → all channels at a specific time point

## 2. Helper Function: Plotting Time and Frequency Domain

We'll use the function below to visualize signals in both the **time domain** and **frequency domain**.  
By default, we only show the frequency range up to 40 Hz (the EEG-relevant range).

In [ ]:
from scipy.fft import rfft, rfftfreq

def plot_time_and_spectrum(signal, sampling_rate, title="", fmax=40):
    import numpy as _np

    if signal.ndim == 1:
        signal = signal[_np.newaxis, :]

    n_channels, n_timepoints = signal.shape
    t = _np.arange(n_timepoints) / sampling_rate

    yf = rfft(signal, axis=1)
    xf = rfftfreq(n_timepoints, 1 / sampling_rate)
    power = _np.abs(yf) ** 2

    fig, axes = plt.subplots(1, 2, figsize=(10, 3))
    offset = 2.0

    for ch in range(n_channels):
        axes[0].plot(t, signal[ch, :] + ch*offset, label=f"Ch {ch+1}", linewidth=1)
    axes[0].set_xlabel("Time [s]")
    axes[0].set_ylabel("Amplitude [a.u.] (offset per channel)")
    axes[0].grid(True, linestyle=":", alpha=0.7)
    axes[0].set_title(f"Time domain {title}")
    if n_channels > 1:
        axes[0].legend()

    for ch in range(n_channels):
        axes[1].plot(xf, power[ch, :], label=f"Ch {ch+1}", linewidth=1)
    axes[1].set_xlim(0, fmax)
    axes[1].set_xlabel("Frequency [Hz]")
    axes[1].set_ylabel("Power [a.u.]")
    axes[1].grid(True, linestyle=":", alpha=0.7)
    axes[1].set_title(f"Frequency domain (0–{fmax} Hz) {title}")
    if n_channels > 1:
        axes[1].legend()

    plt.tight_layout()
    plt.show()

## 3. White Noise

**White noise** is a signal composed of random, *uncorrelated* samples —  
each time point is statistically independent of the previous one.

In the frequency domain, white noise has **equal power at all frequencies** (a flat spectrum).  
That’s why it’s called *white*, by analogy with white light, which contains all colors (frequencies).

Run the code below several times and observe that while the **time-domain signal changes**,  
the **flat shape of the spectrum** remains roughly the same — this is a *statistical property* of the noise.


In [ ]:
white_noise = np.random.randn(time.size)
plot_time_and_spectrum(white_noise, sampling_rate, title="(White Noise)", fmax=40)

## 4. Brown Noise and the 1/f Spectrum

Most physiological signals — including EEG and MEG — show **higher power at low frequencies**  
and **lower power as frequency increases**.  
This overall decline in power is called the **1/f slope**, because power roughly scales with the inverse of frequency.

If we **integrate** white noise (take its cumulative sum), neighboring samples become correlated.  
This produces **Brown noise** (a “random walk”) that drifts more smoothly in time.

In the frequency domain, Brown noise shows a **1/f²** power decay: doubling the frequency makes power about four times smaller.  
Real EEG and LFP signals often show a shallower **1/f** slope, meaning power halves when frequency doubles.  
This pattern reflects the dominance of slow, large-scale neural processes.

Try rerunning the cell below a few times and notice that the **spectrum shape** stays similar  
even though the random signal changes each time.


In [ ]:
brown_noise = np.cumsum(np.random.randn(time.size))
brown_noise -= np.mean(brown_noise)
brown_noise /= np.std(brown_noise)

plot_time_and_spectrum(brown_noise, sampling_rate, title="(Brown Noise)", fmax=40)

## 5. Pure Oscillations: Sine Waves

To create a sine wave for our simulated signal at our sampling rate,  
we use the mathematical definition of a sine wave: `sin(2π × frequency × time)`.

This formula means:
- `frequency` defines how many cycles occur per second (in Hz)  
- `2π` converts cycles into radians (since sine repeats every 2π)  
- `time` is our array of discrete sample points created earlier  

Each point in the sine wave corresponds to one sample at a specific time tick.


In [ ]:
freq = 10  # Hz
sine_wave = np.sin(2 * np.pi * freq * time)
plot_time_and_spectrum(sine_wave, sampling_rate, title="(10 Hz Sine Wave)", fmax=40)

## 6. Interactive Exploration of Sine Waves

Use the sliders to change the **frequency**, **amplitude**, and now also a **DC offset** of a sine wave.

- The **amplitude** scales the oscillation vertically.  
- The **frequency** controls how fast it oscillates.  
- The **offset** shifts the whole wave up or down — adding a constant value.

In the frequency spectrum, this offset appears as a **DC component** (a peak at 0 Hz).

In [ ]:
from ipywidgets import interact

def interactive_sine(freq=10.0, amplitude=1.0, offset=0.0):
    """
    Interactive sine wave with adjustable frequency, amplitude, and DC offset.
    """
    # Generate the signal (keep full precision)
    sine_wave = amplitude * np.sin(2 * np.pi * freq * time) + offset

    # Plot with clean, rounded labels
    plot_time_and_spectrum(
        sine_wave,
        sampling_rate,
        title=f"({freq:.1f} Hz, amp={amplitude:.1f}, offset={offset:.1f})",
        fmax=40
    )

interact(
    interactive_sine,
    freq=(1.0, 40.0, 1.0),
    amplitude=(0.1, 2.0, 0.1),
    offset=(-1.0, 1.0, 0.1)
);


## 7. Combining Multiple Sine Waves Step by Step

Neural signals rarely consist of a single frequency.  
We can simulate more realistic activity by adding several sine waves together — each representing a different neural oscillation.

Each sine wave is defined by two things:
- **Frequency (Hz):** how fast it oscillates.
- **Amplitude:** how strong the oscillation is (scaling the sine wave vertically).

We'll define both in tuples for clarity — one tuple for amplitudes and one for frequencies.  
Then we'll create each sine wave separately and finally sum them together to form a more complex signal.


In [ ]:
# Define amplitudes and frequencies as tuples
amplitudes = (1.0, 0.5, 0.3)
frequencies = (10, 20, 5)

# Create individual sine waves
sine1 = amplitudes[0] * np.sin(2 * np.pi * frequencies[0] * time)
sine2 = amplitudes[1] * np.sin(2 * np.pi * frequencies[1] * time)
sine3 = amplitudes[2] * np.sin(2 * np.pi * frequencies[2] * time)

# Combine them to create a more complex signal
signal_sum = sine1 + sine2 + sine3

plot_time_and_spectrum(signal_sum, sampling_rate, title="(Sum of Three Sine Waves)", fmax=40)

## 7b. Visualizing How Phase Affects Signal Summation

Every sine wave can be described by its **amplitude**, **frequency**, and **phase**.

- **Amplitude** tells us how strong the oscillation is (vertical scaling).  
- **Frequency** tells us how fast it oscillates (cycles per second).  
- **Phase** tells us *where in its cycle* the oscillation starts — its horizontal shift in time.

Two waves can have the **same frequency and amplitude** but start at different phases.  
This offset determines whether they **reinforce** or **cancel** each other when added together.

For example:
- **In phase** (phase = 0): both peaks and troughs align → constructive interference.  
- **Out of phase** (phase = π): peaks of one align with troughs of the other → destructive interference.  
- Intermediate phases cause **partial interference**, where the resulting wave has intermediate amplitude.

Let's visualize this interactively and see how changing the phase of one wave affects their sum.


In [ ]:
def sine_phase_demo(freq1=10.0, freq2=10.0, amp1=1.0, amp2=1.0, phase2=0.0):
    s1 = amp1 * np.sin(2 * np.pi * freq1 * time)
    s2 = amp2 * np.sin(2 * np.pi * freq2 * time + phase2)
    s_sum = s1 + s2

    plt.figure(figsize=(8, 3), dpi=250)
    plt.plot(time, s1, color='tab:blue', alpha=0.6, linewidth=0.9, label="Sine 1")
    plt.plot(time, s2, color='tab:red', alpha=0.6, linewidth=0.9, label="Sine 2 (phase-shifted)")
    plt.plot(time, s_sum, color='k', linewidth=2, label="Sum (resulting waveform)")
    plt.xlim(0, 0.5)
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude [a.u.]")
    plt.title(f"Sum of Two Sine Waves (phase offset = {phase2:.2f} rad)")
    plt.grid(True, linestyle=":", alpha=0.7)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

from ipywidgets import FloatSlider
interact(
    sine_phase_demo,
    freq1=(5.0, 20.0, 1.0),
    freq2=(5.0, 20.0, 1.0),
    amp1=(0.5, 2.0, 0.1),
    amp2=(0.5, 2.0, 0.1),
    phase2=FloatSlider(value=0.0, min=0.0, max=2*np.pi, step=np.pi/12)
);

## Summary

- Represent neural-like time series as arrays (**channels × timepoints**)  
- Create **time vectors** that define discrete sampling points  
- Generate **white** and **brown noise** and understand their **1/f** decay  
- Create and combine **sine waves** (oscillations) with adjustable phase and amplitude  
- Visualize time- and frequency-domain behavior up to 40 Hz  
- Use **interactive sliders** to explore how signal properties change  

*Next session:* we'll extend this to evoked responses, power spectra, and time–frequency analyses.
